In [ ]:
%run cleanData.ipynb
%run cleanData_test.ipynb

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import sys
import os
import random
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier

In [ ]:
#Convert data into float to be used in our algorithm

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

df_train=clean_dataset(df_train)
df_test =clean_dataset(df_test)

In [ ]:
#We are using 80-20 split for train-test
VALID_SIZE = 0.2
#We also use random state for reproducibility
RANDOM_STATE = 2018

train, valid = train_test_split(df_train, test_size=VALID_SIZE, random_state=RANDOM_STATE, shuffle=True )

In [ ]:
df_train

First, we define the predictors variables, then the target variable to predict

In [ ]:
predictors = ['age',
'gender',
'scentLover',
'ecoInterest',
'MrPropre',
'Antikal',
'Ariel',
'Dash',
'pods',
'powder',
'liquid',
'electricToothbrush',
'likesPets',
'hasPet',
'daysSinceActivity',
'nbChildren',
'magasin',
'moyenneSurface',
'superMarket',
'hyperMarket',
'drive',
'hardDiscount']
target = 'washDishes'

In [ ]:
train_X = train[predictors]
train_Y = train[target].values
valid_X = valid[predictors]
valid_Y = valid[target].values


Implementation of Stochastic Gradient Descent Algorithm

In [ ]:
#clf =  SGDClassifier(penalty=None)
clf = SVC()


In [ ]:
#Training of the model
clf.fit(train_X, train_Y)

In [ ]:
#Validation of the model on the remaining 20% of the training set
preds = clf.predict(valid_X)


In [ ]:
preds.shape

In [ ]:
#In order to better understand the importance of each variable, we want to plot the features importance.
def plot_feature_importance():
    tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': clf.feature_importances_})
    tmp = tmp.sort_values(by='Feature importance',ascending=False)
    plt.figure(figsize = (7,4))
    plt.title('Features importance',fontsize=14)
    s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
    s.set_xticklabels(s.get_xticklabels(),rotation=90)
    plt.show()

In [ ]:
plot_feature_importance()

'daysSinceActivity' and 'Age' are the two most important features in the prediction.

Evaluation of our model

In [ ]:
clf.score(train_X, train_Y)
acc = round(clf.score(train_X, train_Y) * 100, 2)
print("SGD accuracy (train set):", acc)

In [ ]:

clf.score(valid_X, valid_Y)
acc = round(clf.score(valid_X, valid_Y) * 100, 2)
print("SGD accuracy (validation set):", acc)

In [ ]:
print(metrics.classification_report(valid_Y, preds, target_names=['Hand', 'Auto']))

In [ ]:
def plot_confusion_matrix():
    cm = pd.crosstab(valid_Y, preds, rownames=['Actual'], colnames=['Predicted'])
    fig, (ax1) = plt.subplots(ncols=1, figsize=(5,5))
    sns.heatmap(cm, 
                xticklabels=['Hand', 'Auto'],
                yticklabels=['Hand', 'Auto'],
                annot=True,ax=ax1,
                linewidths=.2,linecolor="Darkblue", cmap="Blues")
    plt.title('Confusion Matrix', fontsize=14)
    plt.show()

plot_confusion_matrix()

Let's initialize the GradientSearchCV parameters for optimization. We will set only few parameters, as following:

n_estimators: number of trees in the foreset;

max_features: max number of features considered for splitting a node;

max_depth: max number of levels in each decision tree;

min_samples_split: min number of data points placed in a node before the node is split;

min_samples_leaf: min number of data points allowed in a leaf node.

In [ ]:
#Finding best parameters for our SVC model

parameters = {
    'C': [0.8,1.3],
    'kernel':['linear', 'rbf'],
    'gamma' :[0.8, 1.3]
}

#We initialize GridSearchCV with the classifier, the set of parameters, number of folds and also the level of verbose for printing out progress.

grid_svc = GridSearchCV(clf, parameters, scoring='accuracy', cv=10)
grid_svc.fit(train_X, train_Y)

print('Best scores:',grid_svc.best_score_)
print('Best params:',grid_svc.best_params_)




In [ ]:
#Let's run our SVC again with the best parameters.
svc2 = SVC(C = 1.2, gamma =  0.9, kernel= 'rbf')
svc2.fit(train_X, train_Y)
pred_svc2 = svc2.predict(valid_X)
print(classification_report(valid_Y, pred_svc2))

In [ ]:
svc2.score(valid_X, valid_Y)
acc = round(svc2.score(valid_X, valid_Y) * 100, 2)
print("SVC accuracy optimized (validation set):", acc)

In [ ]:
print(metrics.classification_report(valid_Y, preds, target_names=['Hand', 'Auto']))

In [ ]:
Use of our model in the test dataset to submit on Kaggle

In [ ]:
prediction_test = gs_clf.predict(df_test)

In [ ]:
prediction_test[:10]

#As we have drop the userId column, we have to add it again next to the predicted values for the column "washDishes" so we get the userId again from the data set test.
df_test_full = pd.read_csv(r"DS_CentraleSupelec_ST42021/DS_CentraleSupelec_test.csv")

In [ ]:
result_prediction = pd.DataFrame(prediction_test, columns =['WashDishes'])

In [ ]:
result_prediction['WashDishes']

In [ ]:
submit = pd.concat([df_test_full['userId'],result_prediction['WashDishes']], axis=1)
submit.shape

In [ ]:
submit

#Formatting the submit data to fit the submission format expected by Kaggle.
submit.rename(columns={"washDishes": "WashDishes"})

submit['WashDishes'] = submit['WashDishes'].apply(lambda e: 'Auto' if e == 1 else 'Hand')
submit

In [ ]:
submit.to_csv('./DS_CentraleSupelec_ST42021/submit_catboost.csv', index=False)  